In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
!pip install transformers
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00


In [4]:
import gensim
from gensim.models import Word2Vec,KeyedVectors
import pandas as pd
import nltk
import warnings
warnings.filterwarnings('ignore')
import gzip
import shutil
from time import time
from torchvision import transforms
import torch
from PIL import Image


In [5]:
from datasets import load_dataset
dataset=load_dataset("visual_genome", "relationships_v1.0.0")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'image_id', 'url', 'width', 'height', 'coco_id', 'flickr_id', 'relationships'],
        num_rows: 108077
    })
})


In [7]:
class key(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(304,256)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(256,256)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(256,128)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x=self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x




In [8]:
class query(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(304,256)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(256,256)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(256,128)
        self.act_output = nn.Sigmoid()

    def forward(self, y):
        y = self.act1(self.hidden1(y))
        y=self.act2(self.hidden2(y))
        y = self.act_output(self.output(y))
        return y



In [9]:



class CombinedNetwork(nn.Module):
    def __init__(self):
        super(CombinedNetwork, self).__init__()
        self.query_network = query()
        self.key_network = key()
    def forward(self, x):
        # Forward pass through the query and key networks
        output_query = self.query_network(x)
        output_key = self.key_network(x)

        # Multiply the output matrices (element-wise multiplication)
        combined_output = torch.matmul(output_query, output_key.T)
        m=nn.Softmax2D()
        output_1=m(combined_output)



        return output_1



In [10]:

train_data = dataset["train"]
processed_data = []
for entry in train_data:
    if entry["image_id"] is not None and entry["url"] is not None and entry["width"] is not None and entry["height"] is not None and entry["coco_id"] is not None and entry["flickr_id"] is not None and entry["relationships"] is not None:
        processed_entry = {
            "image_id": entry["image_id"],
            "url": entry["url"],
            "width": entry["width"],
            "height": entry["height"],
            "coco_id": entry["coco_id"],
            "flickr_id": entry["flickr_id"],
            "relationships": entry["relationships"]
        }
        processed_data.append(processed_entry)



# Define your batch size
batchsize = 32  # Adjust this according to your needs

# Create a DataLoader for mini-batch processing
dataloader = DataLoader(processed_data, batch_size=batchsize, shuffle=True,drop_last=True)
start=1

for batch in dataloader:





     for desired_image_id in range(start, start+33):
        if desired_image_id == 32:
               start=start+32

    # Change this to the specific image ID you want

# Access the data entry by indexing the "train" subset


# Find the data entry with the desired image ID in the "train" subset
        desired_data_entry = next((entry for entry in batch if entry["image_id"] == desired_image_id), None)

        if desired_data_entry is not None:
    # You've found the data entry
     #print("Found data entry:")
     #print(desired_data_entry)
          relationships = desired_data_entry["relationships"]
          num_relationships = len(relationships)
     #print(num_relationships)
          unique_object_ids = set()
          unique_objects = []
          x=[]
          y=[]
          w=[]
          h=[]

          unique_object_ids.clear()
          unique_objects.clear()  # Clear the set for each new image
          num_relationhips = 0  # Initialize the count to 0 for this image

          for relationship in relationships:
            subject_id = relationship["subject"]["object_id"]
            object_id = relationship["object"]["object_id"]
            obj1 = relationship["subject"]["names"]
            obj2 = relationship["object"]["names"]
            x_sub=relationship["subject"]["x"]
            x_obj=relationship["object"]["x"]
            y_sub=relationship["subject"]["y"]
            y_obj=relationship["object"]["y"]
            w_sub=relationship["subject"]["w"]
            w_obj=relationship["object"]["w"]
            h_sub=relationship["subject"]["h"]
            h_obj=relationship["object"]["h"]
        #print(obj1)
        # Check if the subject and object IDs are not in the set
            if subject_id not in unique_object_ids:
              num_relationships += 1
              unique_object_ids.add(subject_id)
              unique_objects.append(obj1)
              x.append(x_sub)
              y.append(y_sub)
              w.append(w_sub)
              h.append(h_sub)

            if object_id not in unique_object_ids:
              num_relationships += 1
              unique_object_ids.add(object_id)
              unique_objects.append(obj2)
              x.append(x_obj)
              y.append(y_obj)
              w.append(w_obj)
              h.append(h_obj)

        print(f"Image ID {desired_image_id} has {num_relationships} unique objects.")
        num_ob=len(unique_object_ids)
        relationship_matrix = np.zeros((num_relationships, num_relationships), dtype=float)

     #print(zero_array.shape)
        unique_object_ids_list = list(unique_object_ids)
     #unique_obj_list=list(unique_objects)
     #print(num_relationships)
     #print(len(unique_objects))
     #print(len(x))



        for relationship in relationships:   # Set 1 in the matrix for the subject-object relationship
           subject_id = relationship["subject"]["object_id"]
           object_id = relationship["object"]["object_id"]
           subject_index = unique_object_ids_list.index(subject_id)
           object_index = unique_object_ids_list.index(object_id)
           relationship_matrix[subject_index, object_index] = 1.0

        for i in  range(1,num_relationships):
          input_name=unique_objects[i]
          input_x=x[i]
          input_y=y[i]
          input_w=w[i]
          input_h=h[i]
          input=np.array([])
          input=np.append(1,np.array([input_x]))
          input=np.append(1,np.array([input_y]))
          input=np.append(1,np.array([input_w]))
          input=np.append(1,np.array([input_x]))
          input_2d=np.array([])
          np.concatenate((input_2d, input))


        input_tensor = torch.from_numpy(input_2d)
        double_input=input_tensor.double()
        print(double_input.dtype)

       #print(key_name)
       #print(relationship_matrix[subject_index, object_index])
       #print(subject_index)
       #print(object_index)
       #print(relationship_matrix)

       #print(query_name)











        combined_network = CombinedNetwork()
        optimizer = optim.Adam(combined_network.parameters(), lr=0.001)  # You can adjust the learning rate as needed
        criterion = nn.MSELoss()
        num_epochs = 100
        for epoch in range(num_epochs):





            output = combined_network(input_tensor)


            loss = criterion(output, relationship_matrix)


            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
























RuntimeError: ignored

In [ ]:
save_path = 'trained_model.pth'
torch.save(combined_network.state_dict(), save_path)
